In [1]:
import hdbscan
import pickle
import pandas as pd
from typing import Tuple
import pathlib

In [2]:
def import_clusters(
    path_to_tsv: pathlib.Path, path_to_picke: pathlib.Path
) -> Tuple[pd.DataFrame, hdbscan.HDBSCAN]:
    df = pd.read_table(path_to_tsv)
    with open(path_to_picke, "rb") as f:
        return df, pickle.load(f)

In [3]:
df, clusters = import_clusters(
    "../data/output/comparative_analysis/annotated_domains/50000/clusters/clique.clusters.tsv.gz",
    "../data/output/comparative_analysis/annotated_domains/50000/clusters/clique.clusterer.pickle.xz",
)

In [4]:
df1 = df.groupby(["label", "cluster"]).size().to_frame()
df1.columns = ["size"]
df1

size
label     cluster      
MCF10A    -1        761
           0        123
           1         42
           2        245
           3        469
           4        189
MCF10AT1  -1        397
           0         87
           1         57
           2          8
           3        252
           4        274
MCF10CA1a -1        529
           0         44
           1         27
           2         67
           3        314
           4        342